In [1]:
import argparse
import torch
import os
import json
from tqdm import tqdm
import shortuuid
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import math
import torchvision.transforms as T
from torchvision.transforms.functional import InterpolationMode
from transformers import AutoModel, AutoTokenizer
import sys
import re


IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)


def build_transform(input_size):
    MEAN, STD = IMAGENET_MEAN, IMAGENET_STD
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=MEAN, std=STD)
    ])
    return transform


def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio


def dynamic_preprocess(image, min_num=1, max_num=12, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # calculate the existing image aspect ratio
    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if
        i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    # find the closest aspect ratio to the target
    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    # calculate the target width and height
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    # resize the image
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        # split the image
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images


def load_image(image_file, input_size=448, max_num=12, use_dynamic=True):
    image = Image.open(image_file).convert('RGB')
    transform = build_transform(input_size=input_size)
    if use_dynamic:
        images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
        print("Use Dynamic")
    else:
        images = [image]
    pixel_values = [transform(image) for image in images]
    pixel_values = torch.stack(pixel_values)
    return pixel_values


def split_list(lst, n):
    """Split a list into n (roughly) equal-sized chunks"""
    chunk_size = math.ceil(len(lst) / n)  # integer division
    return [lst[i:i+chunk_size] for i in range(0, len(lst), chunk_size)]


def get_chunk(lst, n, k):
    chunks = split_list(lst, n)
    return chunks[k]

/home/zilun/anaconda3/envs/imagerag/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
question_fpath = "/media/zilun/fanxiang4t/GRSM/ImageRAG_git/codebase/inference/MME-RealWorld-RS/MME-RealWorld-Lite.json"
answer_fpath = "/media/zilun/fanxiang4t/GRSM/ImageRAG_git/data/eval/answer_mme-realworld-lite-cot-explicit.jsonl"
model_path = "/media/zilun/fanxiang4t/GRSM/ImageRAG_git/checkpoint/InternVL2_5-8B"
image_root = "/media/zilun/wd-161/datasets/MME-RealWorld/rs_subset/remote_sensing"

In [3]:
answers = [json.loads(line) for line in open(answer_fpath, "r")]
with open(question_fpath, 'r') as file:
    questions = json.load(file)
questions = [question for question in questions if question["Subtask"] == "Remote Sensing"]
questions = get_chunk(questions, 1, 0)
example_answer = answers[0]
print(example_answer)
print(len(questions), len(answers))

{'Question_id': 'perception/remote_sensing/position/2527', 'Image': 'dota_v2_dota_v2_dota_v2_P7036.png', 'Text': 'Where is the yellow crane in the picture?', 'Question Type': 'Multiple Choice', 'Answer choices': ['(A) In the upper right area of the picture', '(B) In the upper left area of the picture', '(C) In the lower right area of the picture', '(D) In the lower left area of the picture', "(E) This image doesn't feature the position."], 'Ground truth': 'A', 'Category': 'position', 'Subtask': 'Remote Sensing', 'Task': 'Perception', 'output': '(C) In the lower right area of the picture\n\nThe yellow crane can be seen in the lower right area of the picture. It is located near the intersection of the roads and is distinguishable by its bright yellow color and crane-like structure.'}
150 150


In [4]:
# Model
model = AutoModel.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True,
    load_in_8bit=True
).eval()
tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    trust_remote_code=True,
    use_fast=False
)

Loading checkpoint shards: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:06<00:00,  1.59s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg

def show_image_with_bbox(image_path, bboxes):
    """
    Display an image with bounding boxes in Jupyter Notebook at its original size.

    Args:
        image_path (str): Path to the image file.
        bboxes (list): List of bounding boxes, each represented as (x1, y1, x2, y2).
    """
    # 读取图片
    img = mpimg.imread(image_path)
    
    # 获取图像的原始尺寸
    img_height, img_width = img.shape[:2]
    
    # 创建图形和轴，设置图形大小为图像的原始尺寸
    dpi = 80  # 可以根据需要调整 DPI
    figsize = (img_width / dpi, img_height / dpi)
    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    
    # 显示图片
    ax.imshow(img)
    
    # 绘制每个边界框
    for bbox in bboxes:
        x1, y1, x2, y2 = bbox
        x1 = int(x1 / 1000 * img_width)
        y1 = int(y1 / 1000 * img_height)
        x2 = int(x2 / 1000 * img_width)
        y2 = int(y2 / 1000 * img_height)
        rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1, linewidth=5, edgecolor='blue', facecolor='none')
        ax.add_patch(rect)
    
    # 移除轴标记
    ax.axis('off')
    
    # 显示图形
    plt.show()

In [6]:
def prepare_input(image_root, model_config, line, test_prompt, use_dynamic=False):
    image_name = line["Image"]
    image_path = os.path.join(image_root, image_name)
    pixel_values = load_image(
        image_path,
        input_size=model_config.vision_config.image_size,
        max_num=model_config.max_dynamic_patch,
        use_dynamic=use_dynamic
    ).to(torch.bfloat16).cuda()

    choices = line['Answer choices']
    image_file = line["Image"]
    qs = line["Text"]
    # choice_prompt = ' The choices are listed below: \n'
    # for choice in choices:
    #     choice_prompt += choice + "\n"
    # qs += choice_prompt + test_prompt + '\nThe best answer is:'
    # final_instruction = qs
    final_instruction = test_prompt + "<ref>" + "yellow crane" + "</ref>"
    return image_path, pixel_values, [pixel_values.size(0)], final_instruction

In [7]:
def inference(model, tokenizer, image_tensors, prompt, generation_param):
    num_patches_list = [image_tensors.shape[0]]
    prompts = [prompt]
    generation_config = dict(
        max_new_tokens=generation_param["max_new_tokens"],
        do_sample=True if generation_param["temperature"] > 0 else False,
        temperature=generation_param["temperature"],
        top_p=generation_param["top_p"],
        num_beams=generation_param["num_beams"],
    )
    with torch.inference_mode():
        responses = model.batch_chat(
            tokenizer,
            image_tensors,
            num_patches_list=num_patches_list,
            questions=prompts,
            generation_config=generation_config
    )
    return responses

In [8]:
# test_prompt = "Select the best answer to the above multiple-choice question based on the image. Respond with only the letter (A, B, C, D, or E) of the correct option."
test_prompt = "<image>\nPlease provide the bounding box coordinate of the region this sentence describes: "
image_path, image_tensors, num_patches_list, qs = prepare_input(image_root, model.config, example_answer, test_prompt, use_dynamic=True)
# image_path = "/home/zilun/Desktop/953457412.jpg"

Use Dynamic


In [9]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [10]:
print(image_tensors.shape)
print(num_patches_list)
print(qs)
print("GT: {}\n".format(example_answer["Ground truth"]))

generation_param_dict = {
    "max_new_tokens": 128,
    "temperature": 0.2,
    "top_p": None,
    "num_beams": 1,
}

responses = inference(model, tokenizer, image_tensors, qs, generation_param_dict)
print("Ouput: {}".format(responses[0]))

torch.Size([7, 3, 448, 448])
[7]
<image>
Please provide the bounding box coordinate of the region this sentence describes: <ref>yellow crane</ref>
GT: A



/home/zilun/anaconda3/envs/imagerag/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Ouput: yellow crane[[721, 315, 1000, 715]]


In [1]:
# PATTERN = re.compile(r'\[*\[(.*?),(.*?),(.*?),(.*?)\]\]*')
# predict_bbox = re.findall(PATTERN, responses[0])
# try:
#     predict_bbox = (float(predict_bbox[0][0]), float(predict_bbox[0][1]), float(predict_bbox[0][2]),
#                     float(predict_bbox[0][3]))
# except:
#     predict_bbox = None
# bboxes = [predict_bbox]
# print(predict_bbox)
# show_image_with_bbox(image_path, bboxes)

In [4]:
def extract_characters_regex(s, choices):
    s = s.strip()
    answer_prefixes = [
        "The best answer is",
        "The correct answer is",
        "The answer is",
        "The answer",
        "The best option is"
        "The correct option is",
        "Best answer:"
        "Best option:",
    ]
    for answer_prefix in answer_prefixes:
        s = s.replace(answer_prefix, "")

    if len(s.split()) > 10 and not re.search("[ABCDE]", s):
        return ""
    matches = re.search(r'[ABCDE]', s)
    if matches is None:
        for choice in choices:
            if s.lower() in choice.lower():
                return choice[1]
        return ""
    return matches[0]